# Lab: Chicago taxifare tip prediction on Google Cloud Vertex Pipelines using the TFX SDK

## Learning objectives

* Define a machine learning pipeline to predict taxi fare tips using the TFX SDK.
* Compile and run a TFX pipeline on Google Cloud's Vertex Pipelines.

## Dataset

The [Chicago Taxi Trips](https://pantheon.corp.google.com/marketplace/details/city-of-chicago-public-data/chicago-taxi-trips) dataset is one of the [public datasets hosted with BigQuery](https://cloud.google.com/bigquery/public-data/), which includes taxi trips from 2013 to the present, reported to the City of Chicago in its role as a regulatory agency. The task is to predict whether a given trip will result in a tip > 20%.

## Setup

### Define constants

In [1]:
GOOGLE_CLOUD_PROJECT_ID = !(gcloud config get-value core/project)
GOOGLE_CLOUD_PROJECT_ID = GOOGLE_CLOUD_PROJECT_ID[0]

In [2]:
GOOGLE_CLOUD_REGION = 'us-central1'

In [3]:
BQ_DATASET_NAME = 'chicago_taxifare_tips'
BQ_TABLE_NAME = 'chicago_taxi_tips_ml'
BQ_LOCATION = 'US'
BQ_URI = f"bq://{GOOGLE_CLOUD_PROJECT_ID}.{BQ_DATASET_NAME}.{BQ_TABLE_NAME}"

In [169]:
DATASET_DISPLAY_NAME = 'chicago-taxifare-tips'
MODEL_DISPLAY_NAME = f'{DATASET_DISPLAY_NAME}-classifier'
PIPELINE_NAME = f'{MODEL_DISPLAY_NAME}-train-pipeline'

### Create Google Cloud Storage bucket for storing Vertex Pipeline artifacts

In [6]:
GCS_LOCATION = f"gs://{GOOGLE_CLOUD_PROJECT_ID}-tfx"

In [8]:
!gsutil mb -l $GOOGLE_CLOUD_REGION $GCS_LOCATION

Creating gs://dougkelly-vertex-demos-tfx/...
ServiceException: 409 A Cloud Storage bucket named 'dougkelly-vertex-demos-tfx' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


### Import libraries

In [9]:
import os
import tensorflow as tf
import tfx
import kfp

from google.cloud import bigquery
from google.cloud import aiplatform as vertex_ai

In [10]:
print(f"tensorflow: {tf.__version__}")
print(f"tfx: {tfx.__version__}")
print(f"kfp: {kfp.__version__}")
print(f"Google Cloud Vertex AI Python SDK: {vertex_ai.__version__}")

tensorflow: 2.6.2
tfx: 1.4.0
kfp: 1.8.1
Google Cloud Vertex AI Python SDK: 1.7.1


## Create BigQuery dataset

In [11]:
!bq --location=$BQ_LOCATION mk -d \
$GOOGLE_CLOUD_PROJECT_ID:$BQ_DATASET_NAME

Dataset 'dougkelly-vertex-demos:chicago_taxifare_tips' successfully created.


## Create BigQuery dataset for ML classification task

In [12]:
SAMPLE_SIZE = 20000
YEAR = 2020

In [85]:
sql_script = '''
CREATE OR REPLACE TABLE `@PROJECT_ID.@DATASET.@TABLE` 
AS (
    WITH
      taxitrips AS (
      SELECT
        trip_start_timestamp,
        trip_seconds,
        trip_miles,
        payment_type,
        pickup_longitude,
        pickup_latitude,
        dropoff_longitude,
        dropoff_latitude,
        tips,
        fare
      FROM
        `bigquery-public-data.chicago_taxi_trips.taxi_trips`
      WHERE 1=1 
      AND pickup_longitude IS NOT NULL
      AND pickup_latitude IS NOT NULL
      AND dropoff_longitude IS NOT NULL
      AND dropoff_latitude IS NOT NULL
      AND trip_miles > 0
      AND trip_seconds > 0
      AND fare > 0
      AND EXTRACT(YEAR FROM trip_start_timestamp) = @YEAR
    )

    SELECT
      trip_start_timestamp,
      EXTRACT(MONTH from trip_start_timestamp) as trip_month,
      EXTRACT(DAY from trip_start_timestamp) as trip_day,
      EXTRACT(DAYOFWEEK from trip_start_timestamp) as trip_day_of_week,
      EXTRACT(HOUR from trip_start_timestamp) as trip_hour,
      trip_seconds,
      trip_miles,
      payment_type,
      ST_AsText(
          ST_SnapToGrid(ST_GeogPoint(pickup_longitude, pickup_latitude), 0.1)
      ) AS pickup_grid,
      ST_AsText(
          ST_SnapToGrid(ST_GeogPoint(dropoff_longitude, dropoff_latitude), 0.1)
      ) AS dropoff_grid,
      ST_Distance(
          ST_GeogPoint(pickup_longitude, pickup_latitude), 
          ST_GeogPoint(dropoff_longitude, dropoff_latitude)
      ) AS euclidean,
      CONCAT(
          ST_AsText(ST_SnapToGrid(ST_GeogPoint(pickup_longitude,
              pickup_latitude), 0.1)), 
          ST_AsText(ST_SnapToGrid(ST_GeogPoint(dropoff_longitude,
              dropoff_latitude), 0.1))
      ) AS loc_cross,
      IF((tips/fare >= 0.2), 1, 0) AS tip_bin,
      IF(ABS(MOD(FARM_FINGERPRINT(STRING(trip_start_timestamp)), 10)) < 9, 'UNASSIGNED', 'TEST') AS ml_use
    FROM
      taxitrips
    LIMIT @LIMIT
)
'''

In [86]:
sql_script = sql_script.replace(
    '@PROJECT_ID', GOOGLE_CLOUD_PROJECT_ID).replace(
    '@DATASET', BQ_DATASET_NAME).replace(
    '@TABLE', BQ_TABLE_NAME).replace(
    '@YEAR', str(YEAR)).replace(
    '@LIMIT', str(SAMPLE_SIZE))

In [87]:
bq_client = bigquery.Client(project=GOOGLE_CLOUD_PROJECT_ID, location=BQ_LOCATION)
job = bq_client.query(sql_script)
_ = job.result()

In [90]:
%%bigquery

SELECT ml_use, COUNT(*)
FROM chicago_taxifare_tips.chicago_taxi_tips_ml
GROUP BY ml_use

Downloading: 100%|██████████| 2/2 [00:01<00:00,  1.49rows/s]


,ml_use,f0_
0,UNASSIGNED,18127
1,TEST,1873


## Create a Vertex AI managed dataset resource for pipeline dataset lineage tracking

### Initialize Vertex AI Python SDK

In [91]:
vertex_ai.init(project=GOOGLE_CLOUD_PROJECT_ID, location=GOOGLE_CLOUD_REGION)

### Create Vertex managed tabular dataset

In [94]:
tabular_dataset = vertex_ai.TabularDataset.create(display_name=f"{DATASET_DISPLAY_NAME}", bq_source=f"{BQ_URI}")
tabular_dataset.gca_resource

INFO:google.cloud.aiplatform.datasets.dataset:Creating TabularDataset
INFO:google.cloud.aiplatform.datasets.dataset:Create TabularDataset backing LRO: projects/617979904441/locations/us-central1/datasets/4914403559786676224/operations/8192561262836580352
INFO:google.cloud.aiplatform.datasets.dataset:TabularDataset created. Resource name: projects/617979904441/locations/us-central1/datasets/4914403559786676224
INFO:google.cloud.aiplatform.datasets.dataset:To use this TabularDataset in another session:
INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.TabularDataset('projects/617979904441/locations/us-central1/datasets/4914403559786676224')


name: "projects/617979904441/locations/us-central1/datasets/4914403559786676224"
display_name: "chicago-taxifare-tips"
metadata_schema_uri: "gs://google-cloud-aiplatform/schema/dataset/metadata/tabular_1.0.0.yaml"
create_time {
  seconds: 1638842506
  nanos: 675382000
}
update_time {
  seconds: 1638842507
  nanos: 274500000
}
etag: "AMEw9yMUktBa4vLbkjJG08QJYd65MitZkjzr9qSHPVOlHLazO097R4ZOem8WwjyM1psz"
labels {
  key: "aiplatform.googleapis.com/dataset_metadata_schema"
  value: "TABLE"
}
metadata {
  struct_value {
    fields {
      key: "inputConfig"
      value {
        struct_value {
          fields {
            key: "bigquerySource"
            value {
              struct_value {
                fields {
                  key: "uri"
                  value {
                    string_value: "bq://dougkelly-vertex-demos.chicago_taxifare_tips.chicago_taxi_tips_ml"
                  }
                }
              }
            }
          }
        }
      }
    }
  }
}

## Create a TFX pipeline

In [20]:
PIPELINE_DIR="tfx_taxifare_tips"

### Write model code

In [ ]:
%%writefile {PIPELINE_DIR}/model_training/features.py


In [ ]:
%%writefile {PIPELINE_DIR}/model_training/preprocessing.py


In [ ]:
%%writefile {PIPELINE_DIR}/model_training/model.py


### Write pipeline definition with the TFX SDK

In [ ]:
%%writefile {PIPELINE_DIR}/pipeline.py


In [ ]:
%%writefile {PIPELINE_DIR}/runner.py


## Run your TFX pipeline on Vertex Pipelines

### Create a Artifact Registry on Google Cloud for your pipeline container image

In [155]:
ARTIFACT_REGISTRY="tfx-taxifare-tips"

In [96]:
# TODO: create a Docker Artifact Registry using the gcloud CLI.
# Documentation link: https://cloud.google.com/sdk/gcloud/reference/artifacts/repositories/create

!gcloud artifacts repositories create {ARTIFACT_REGISTRY} \
--repository-format=docker \
--location={GOOGLE_CLOUD_REGION} \
--description="Artifact registry for TFX pipeline images for Chicago taxifare prediction."

ERROR: (gcloud.artifacts.repositories.create) ALREADY_EXISTS: the repository already exists


In [156]:
IMAGE_NAME="tfx-taxifare-tips"
IMAGE_TAG="latest"
IMAGE_URI=f"{GOOGLE_CLOUD_REGION}-docker.pkg.dev/{GOOGLE_CLOUD_PROJECT_ID}/{ARTIFACT_REGISTRY}/{IMAGE_NAME}:{IMAGE_TAG}"

### Set the pipeline configurations for the Vertex AI run

In [234]:
os.environ["DATASET_DISPLAY_NAME"] = DATASET_DISPLAY_NAME
os.environ["MODEL_DISPLAY_NAME"] = MODEL_DISPLAY_NAME
os.environ["PIPELINE_NAME"] = PIPELINE_NAME
os.environ["GOOGLE_CLOUD_PROJECT_ID"] = GOOGLE_CLOUD_PROJECT_ID
os.environ["GOOGLE_CLOUD_REGION"] = GOOGLE_CLOUD_REGION
os.environ["GCS_LOCATION"] = GCS_LOCATION
os.environ["TRAIN_LIMIT"] = "5000"
os.environ["TEST_LIMIT"] = "1000"
os.environ["BEAM_RUNNER"] = "DataflowRunner"
os.environ["TRAINING_RUNNER"] = "vertex"
os.environ["TFX_IMAGE_URI"] = IMAGE_URI
os.environ["ENABLE_CACHE"] = "1"

In [235]:
from tfx_taxifare_tips.tfx_pipeline import config
import importlib
importlib.reload(config)

for key, value in config.__dict__.items():
    if key.isupper(): print(f'{key}: {value}')

GOOGLE_CLOUD_PROJECT_ID: dougkelly-vertex-demos
GOOGLE_CLOUD_REGION: us-central1
GCS_LOCATION: gs://dougkelly-vertex-demos-tfx
ARTIFACT_STORE_URI: gs://dougkelly-vertex-demos-tfx/tfx-artifacts
MODEL_REGISTRY_URI: gs://dougkelly-vertex-demos-tfx/model-registry
DATASET_DISPLAY_NAME: chicago-taxifare-tips
MODEL_DISPLAY_NAME: chicago-taxifare-tips-classifier
PIPELINE_NAME: chicago-taxifare-tips-classifier-train-pipeline
ML_USE_COLUMN: ml_use
EXCLUDE_COLUMNS: trip_start_timestamp
TRAIN_LIMIT: 5000
TEST_LIMIT: 1000
SERVE_LIMIT: 0
NUM_TRAIN_SPLITS: 4
NUM_EVAL_SPLITS: 1
ACCURACY_THRESHOLD: 0.8
USE_KFP_SA: False
TFX_IMAGE_URI: us-central1-docker.pkg.dev/dougkelly-vertex-demos/tfx-taxifare-tips/tfx-taxifare-tips:latest
BEAM_RUNNER: DataflowRunner
BEAM_DIRECT_PIPELINE_ARGS: ['--project=dougkelly-vertex-demos', '--temp_location=gs://dougkelly-vertex-demos-tfx/temp']
BEAM_DATAFLOW_PIPELINE_ARGS: ['--project=dougkelly-vertex-demos', '--temp_location=gs://dougkelly-vertex-demos-tfx/temp', '--region=u

### Build the TFX pipeline container image

In [254]:
!echo $TFX_IMAGE_URI

us-central1-docker.pkg.dev/dougkelly-vertex-demos/tfx-taxifare-tips/tfx-taxifare-tips:latest


In [255]:
# !docker build . -t test-image

In [256]:
!gcloud builds submit --tag $TFX_IMAGE_URI . --timeout=20m --machine-type=e2-highcpu-8

Creating temporary tarball archive of 26 file(s) totalling 267.1 KiB before compression.
Uploading tarball of [.] to [gs://dougkelly-vertex-demos_cloudbuild/source/1639112263.267746-dc49aada98f941ca8a3d522f7e57a263.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/dougkelly-vertex-demos/locations/global/builds/2b8a42f1-fa91-43ee-b1ac-3269318afa84].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/2b8a42f1-fa91-43ee-b1ac-3269318afa84?project=617979904441].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "2b8a42f1-fa91-43ee-b1ac-3269318afa84"

FETCHSOURCE
Fetching storage object: gs://dougkelly-vertex-demos_cloudbuild/source/1639112263.267746-dc49aada98f941ca8a3d522f7e57a263.tgz#1639112263639872
Copying gs://dougkelly-vertex-demos_cloudbuild/source/1639112263.267746-dc49aada98f941ca8a3d522f7e57a263.tgz#1639112263639872...
/ [1 files][ 48.4 KiB/ 48.4 KiB]                                                
Ope

### Compile the TFX pipeline

In [249]:
import tfx_taxifare_tips

In [260]:
# importlib.reload(tfx_taxifare_tips)

INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/617979904441/locations/us-central1/pipelineJobs/chicago-taxifare-tips-classifier-train-pipeline-20211210050353
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/617979904441/locations/us-central1/pipelineJobs/chicago-taxifare-tips-classifier-train-pipeline-20211210050353')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/chicago-taxifare-tips-classifier-train-pipeline-20211210050353?project=617979904441
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/617979904441/locations/us-central1/pipelineJobs/chicago-taxifare-tips-classifier-train-pipeline-20211210050353 current state:
PipelineState.PIPELINE_STATE_PENDING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJo

In [257]:
PIPELINE_DEFINITION_FILE = f'{config.PIPELINE_NAME}.json'

In [258]:
from tfx_taxifare_tips.tfx_pipeline import pipeline_runner

pipeline_definition = pipeline_runner.compile_training_pipeline(PIPELINE_DEFINITION_FILE)

INFO:root:pipeline_root: gs://dougkelly-vertex-demos-tfx/tfx-artifacts/chicago-taxifare-tips-classifier-train-pipeline
INFO:root:tfx_image_uri: us-central1-docker.pkg.dev/dougkelly-vertex-demos/tfx-taxifare-tips/tfx-taxifare-tips:latest
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:root:Pipeline components: TrainDataGen, TestDataGen, SchemaImporter, StatisticsGen, ExampleValidator, Tranform, ModelTrainer, BaselineModelResolver, ModelEvaluator, ModelPusher
INFO:root:Beam pipeline args: ['--project=dougkelly-vertex-demos', '--temp_location=gs://dougkelly-vertex-demos-tfx/temp', '--region=us-central1', '--runner=DataflowRunner']
INFO:root:pipeline_info: PipelineInfo(pipeline_name: chicago-taxifare-tips-classifier-train-pipeline, pipeline_root: gs://dougkelly-vertex-demos-tfx/tfx-artifacts/chicago-taxifare-tips-classifier-train-pipeline, run_id: None)


In [259]:
pipeline_job = vertex_ai.pipeline_jobs.PipelineJob(
          display_name=config.PIPELINE_NAME,
          template_path=PIPELINE_DEFINITION_FILE,
          pipeline_root=os.path.join(config.ARTIFACT_STORE_URI,config.PIPELINE_NAME)
      )
pipeline_job.run(sync=False)

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob


### Extracting pipeline run metadata

In [175]:
pipeline_df = vertex_ai.get_pipeline_df(PIPELINE_NAME)
pipeline_df = pipeline_df[pipeline_df.pipeline_name == PIPELINE_NAME]
pipeline_df.T

,0,1,2
pipeline_name,chicago-taxifare-tips-classifier-train-pipeline,chicago-taxifare-tips-classifier-train-pipeline,chicago-taxifare-tips-classifier-train-pipeline
run_name,chicago-taxifare-tips-classifier-train-pipelin...,chicago-taxifare-tips-classifier-train-pipelin...,chicago-taxifare-tips-classifier-train-pipelin...


INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/617979904441/locations/us-central1/pipelineJobs/chicago-taxifare-tips-classifier-train-pipeline-20211208235005 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/617979904441/locations/us-central1/pipelineJobs/chicago-taxifare-tips-classifier-train-pipeline-20211208235005 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/617979904441/locations/us-central1/pipelineJobs/chicago-taxifare-tips-classifier-train-pipeline-20211208235005 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/617979904441/locations/us-central1/pipelineJobs/chicago-taxifare-tips-classifier-train-pipeline-20211208235005 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/617979904441/locations/us-ce

### Upload trained model from Google Cloud Storage to Vertex AI

In [228]:
"""Pipeline definition code."""
import os
import sys
import logging
from typing import Text

import tensorflow_model_analysis as tfma
from tfx.proto import example_gen_pb2, transform_pb2, pusher_pb2
from tfx.v1.types.standard_artifacts import Model, ModelBlessing, Schema
from tfx.v1.extensions.google_cloud_big_query import BigQueryExampleGen
from tfx.v1.extensions.google_cloud_ai_platform import Trainer as VertexTrainer
from tfx.v1.dsl import Pipeline, Importer, Resolver, Channel
from tfx.v1.dsl.experimental import LatestBlessedModelStrategy
from tfx.v1.components import (
    StatisticsGen,
    ExampleValidator,
    Transform,
    Evaluator,
    Pusher,
)

from tfx_taxifare_tips.tfx_pipeline import config
from tfx_taxifare_tips.model_training import features, bq_datasource_utils

In [135]:
import os, time
from tfx.orchestration.experimental.interactive.interactive_context import (
    InteractiveContext,
)
ARTIFACT_STORE = os.path.join(os.sep, "home", "jupyter", "artifact-store")
SERVING_MODEL_DIR = os.path.join(os.sep, "home", "jupyter", "serving_model")
DATA_ROOT = "../../../data"

PIPELINE_NAME = "tfx-covertype-classifier"
PIPELINE_ROOT = os.path.join(
    ARTIFACT_STORE, PIPELINE_NAME, time.strftime("%Y%m%d_%H%M%S")
)

os.makedirs(PIPELINE_ROOT, exist_ok=True)

context = InteractiveContext(
    pipeline_name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
    metadata_connection_config=None,
)

In [229]:
import_schema = Importer(
    source_uri="tfx_taxifare_tips/raw_schema",
    artifact_type=Schema,
).with_id("SchemaImporter")

In [230]:
context.run(import_schema)

INFO:absl:Running driver for SchemaImporter
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Processing source uri: tfx_taxifare_tips/raw_schema, properties: {}, custom_properties: {}
INFO:absl:Reusing existing artifact
INFO:absl:Running executor for SchemaImporter
INFO:absl:Running publisher for SchemaImporter
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaImporter
    execution_id: 2
    outputs:
        result: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "tfx_taxifare_tips/raw_schema"
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.4.0"
          }
        }
        create_time_since_epoch: 1638984854306
        last_update_time_since_epoch: 1638984854306
        , artifact_type: id: 14
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [231]:
import_schema.outputs["result"].get()[0].uri

'tfx_taxifare_tips/raw_schema'

In [ ]:
    examplevalidator = ExampleValidator(
        statistics=statisticsgen.outputs["statistics"],
        schema=import_schema.outputs["result"],
    ).with_id("ExampleValidator")